In [72]:
!pip install -q dspy-ai beautifulsoup4 requests pandas tqdm lxml groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.76.0 requires grpcio>=1.76.0, but you have grpcio 1.67.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.2 which is incompatible.


In [73]:
import os
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import dspy


In [75]:
import dspy
import os


os.environ["GROQ_API_KEY"] = "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"

lm = dspy.LM(
    model="groq/llama-3.1-8b-instant",
    api_key=os.environ["GROQ_API_KEY"],
    max_tokens=180,
    temperature=0.3,
)
dspy.settings.configure(lm=lm)



In [76]:
import dspy

class Ping(dspy.Signature):
    text: str = dspy.InputField()
    out: str = dspy.OutputField()

ping = dspy.Predict(Ping)
print(ping(text="Say OK").out)


OK


In [77]:
URLS = [
    "https://en.wikipedia.org/wiki/Sustainable_agriculture",
    "https://www.nature.com/articles/d41586-025-03353-5",
    "https://www.sciencedirect.com/science/article/pii/S1043661820315152",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/",
    "https://www.fao.org/3/y4671e/y4671e06.htm",
    "https://www.medscape.com/viewarticle/time-reconsider-tramadol-chronic-pain-2025a1000ria",
    "https://www.sciencedirect.com/science/article/pii/S0378378220307088",
    "https://www.frontiersin.org/news/2025/09/01/rectangle-telescope-finding-habitable-planets",
    "https://www.medscape.com/viewarticle/second-dose-boosts-shingles-protection-adults-aged-65-years-2025a1000ro7",
    "https://www.theguardian.com/global-development/2025/oct/13/astro-ambassadors-stargazers-himalayas-hanle-ladakh-india",
]


In [78]:
def scrape_url(url, max_chars=2500):
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers, timeout=20)
    soup = BeautifulSoup(r.text, "lxml")

    for tag in soup(["script", "style", "nav", "footer", "aside"]):
        tag.decompose()

    text = " ".join(p.get_text(" ", strip=True) for p in soup.find_all("p"))
    text = re.sub(r"\s+", " ", text)

    return text[:max_chars]


In [79]:
class ExtractEntities(dspy.Signature):
    """
    Extract key entities.

    Output format (ONE PER LINE):
    entity | type
    """
    paragraph: str = dspy.InputField()
    entities_text: str = dspy.OutputField()
entity_extractor = dspy.Predict(ExtractEntities)

In [80]:
def extract_entities(text):
    resp = entity_extractor(paragraph=text).entities_text
    entities = []

    for line in resp.splitlines():
        if "|" not in line:
            continue
        entity, typ = line.split("|", 1)
        entities.append({
            "entity": entity.strip(),
            "type": typ.strip()
        })
    seen = {}
    for e in entities:
        seen[e["entity"].lower()] = e

    return list(seen.values())


In [81]:
class ExtractTriples(dspy.Signature):
    """
    Extract relationships.
    Output format:
    source -- relation --> target
    """
    paragraph: str = dspy.InputField()
    triples_text: str = dspy.OutputField()
triple_extractor = dspy.Predict(ExtractTriples)


In [82]:
def extract_triples(text, entities):
    resp = triple_extractor(paragraph=text).triples_text
    triples = []

    for line in resp.splitlines():
        if "--" in line and "-->" in line:
            try:
                src, rest = line.split("--", 1)
                rel, tgt = rest.split("-->", 1)
                triples.append({
                    "source": src.strip(),
                    "relation": rel.strip()[:40],
                    "target": tgt.strip()
                })
            except:
                continue

    return triples


In [83]:
def clean(x):
    return re.sub(r"[^a-zA-Z0-9_]", "_", x)

def triples_to_mermaid(triples, entities):
    entity_set = {e["entity"].lower() for e in entities}
    lines = ["graph TD"]

    for t in triples:
        if t["source"].lower() in entity_set and t["target"].lower() in entity_set:
            lines.append(
                f"  {clean(t['source'])} -- {t['relation']} --> {clean(t['target'])}"
            )
    return "\n".join(lines)

In [62]:
class EntityWithAttr(BaseModel):
    entity: str
    attr_type: str


class ExtractEntities(dspy.Signature):
    """Return ONLY valid JSON array of objects with:
    - entity
    - attr_type
    No partial objects.
    """

    paragraph: str = dspy.InputField()
    entities: list[EntityWithAttr] = dspy.OutputField()


extractor = dspy.Predict(
    ExtractEntities,
    max_tokens=800
)
def extract_entities(text: str):
    ents = extractor(paragraph=text).entities
    return [e for e in ents if e.entity and e.attr_type]

In [84]:
rows = []

for idx, url in enumerate(tqdm(URLS)):
    text = scrape_url(url)

    entities = extract_entities(text)
    time.sleep(4)

    triples = extract_triples(text, entities)
    time.sleep(4)

    mermaid = triples_to_mermaid(triples, entities)

    with open(f"mermaid_{idx+1}.md", "w") as f:
        f.write(mermaid)

    for e in entities:
        rows.append({
            "link": url,
            "tag": e["entity"],
            "tag_type": e["type"]
        })
    time.sleep(6)   # CRITICAL for Groq TPM

  0%|          | 0/10 [00:00<?, ?it/s]2025/12/17 06:41:03 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=180. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.3)  if the reason for truncation is repetition.
2025/12/17 06:41:08 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=180. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.3)  if the reason for truncation is repetition.
 10%|█         | 1/10 [00:16<02:25, 16.19s/it]2025/12/17 06:41:20 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=180. You can inspect the latest LM interactions with `dspy.inspect_history(

In [85]:
df = pd.DataFrame(rows).drop_duplicates()
df.to_csv("tags.csv", index=False)
df.head()

,link,tag,tag_type
0,https://en.wikipedia.org/wiki/Sustainable_agri...,society,organization
1,https://en.wikipedia.org/wiki/Sustainable_agri...,food,product
2,https://en.wikipedia.org/wiki/Sustainable_agri...,textile,product
3,https://en.wikipedia.org/wiki/Sustainable_agri...,ecosystem,concept
4,https://en.wikipedia.org/wiki/Sustainable_agri...,methods,concept


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('tag').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('tag_type').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['tag_type'].value_counts()
    for x_label, grp in _df_2.groupby('tag')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('tag')
_ = plt.ylabel('tag_type')